<a href="https://colab.research.google.com/github/prof-sd1/SCD/blob/main/smart_city_dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Smart City Dashboard


In [1]:
!pip install -q plotly pandas numpy scikit-learn streamlit pyngrok requests geopandas folium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 75.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 97.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.0 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import requests
import time
from datetime import datetime, timedelta
import folium
from folium import plugins
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import streamlit as st
import threading
from pyngrok import ngrok

In [4]:
# 🔑 Replace with your OpenWeatherMap API key
API_KEY = "b4f73b0c7123cf33a1a24cd995f6b88a"  # Sign up at https://home.openweathermap.org/api_keys

CITIES = ["Addis Ababa","New York", "Los Angeles", "Chicago", "Houston", "Phoenix"]

def fetch_air_quality(city):
    try:
        url = f"http://api.openweathermap.org/data/2.5/air_pollution?appid={API_KEY}&q={city}"
        response = requests.get(url).json()
        data = response['list'][0]
        return {
            "city": city,
            "aqi": data["main"]["aqi"],
            "pm2_5": data["components"]["pm2_5"],
            "pm10": data["components"]["pm10"],
            "no2": data["components"]["no2"],
            "timestamp": datetime.now()
        }
    except:
        print(f"Failed to fetch real data for {city}, using synthetic fallback")
        return {
            "city": city,
            "aqi": np.random.randint(1, 6),
            "pm2_5": np.random.uniform(5, 100),
            "pm10": np.random.uniform(10, 150),
            "no2": np.random.uniform(10, 80),
            "timestamp": datetime.now()
        }

# Fetch air quality
air_quality_data = [fetch_air_quality(city) for city in CITIES]
df_air = pd.DataFrame(air_quality_data)

Failed to fetch real data for Addis Ababa, using synthetic fallback
Failed to fetch real data for New York, using synthetic fallback
Failed to fetch real data for Los Angeles, using synthetic fallback
Failed to fetch real data for Chicago, using synthetic fallback
Failed to fetch real data for Houston, using synthetic fallback
Failed to fetch real data for Phoenix, using synthetic fallback


In [6]:
def fetch_traffic(city):
    try:
        # Example TomTom endpoint (needs API key)
        url = f"https://api.tomtom.com/traffic/services/4/flowSegmentData/absolute/10/json?key={e9eQ9m19zNwNDmH1CJ1Pzvc0wmhuZDvx}&point={lat},{lon}"
        # For demo, use synthetic
        raise Exception("No API key")
    except:
        congestion = np.random.uniform(0.2, 0.95)
        speed = np.random.uniform(10, 60)
        delay = np.random.uniform(0, 30)
        return {
            "city": city,
            "congestion_level": round(congestion, 2),
            "avg_speed_kmh": round(speed, 2),
            "delay_minutes": round(delay, 2),
            "timestamp": datetime.now()
        }

traffic_data = [fetch_traffic(city) for city in CITIES]
df_traffic = pd.DataFrame(traffic_data)

In [7]:
np.random.seed(42)
health_data = []
for city in CITIES:
    for _ in range(30):  # 30 days
        date = datetime.now() - timedelta(days=np.random.randint(0, 30))
        visits = np.random.poisson(50) + (df_air[df_air.city==city].pm2_5.values[0] * 0.5)  # pollution link
        health_data.append({
            "city": city,
            "date": date,
            "emergency_visits": int(visits),
            "respiratory_cases": int(visits * 0.3 + np.random.rand() * 5)
        })

df_health = pd.DataFrame(health_data)
df_health['date'] = pd.to_datetime(df_health['date'])

In [8]:
noise_data = []
for city in CITIES:
    for hour in range(24):
        for _ in range(5):  # 5 sensors per city
            noise = np.random.normal(70, 15)  # dB
            noise = max(40, min(noise, 120))  # realistic range
            noise_data.append({
                "city": city,
                "hour": hour,
                "noise_dB": round(noise, 2),
                "location_lat": np.random.uniform(33, 41),
                "location_lon": np.random.uniform(-118, -73)
            })

df_noise = pd.DataFrame(noise_data)

In [9]:
edu_data = []
for city in CITIES:
    schools = np.random.randint(200, 1000)
    students_per_school = np.random.randint(300, 800)
    budget_per_student = np.random.randint(8000, 15000)
    graduation_rate = np.random.uniform(0.7, 0.95)
    edu_data.append({
        "city": city,
        "total_schools": schools,
        "total_students": schools * students_per_school,
        "budget_per_student": budget_per_student,
        "graduation_rate": round(graduation_rate, 3)
    })

df_edu = pd.DataFrame(edu_data)

In [10]:
crime_data = []
for city in CITIES:
    for day_offset in range(30):
        date = datetime.now() - timedelta(days=day_offset)
        crimes = np.random.poisson(15)
        crime_data.append({
            "city": city,
            "date": date,
            "crimes_reported": crimes,
            "violent_crimes": np.random.binomial(crimes, 0.2)
        })

df_crime = pd.DataFrame(crime_data)
df_crime['date'] = pd.to_datetime(df_crime['date'])

In [11]:
# Merge key city metrics
df_summary = df_air[["city", "aqi", "pm2_5", "pm10"]].copy()
df_summary = df_summary.merge(df_traffic[["city", "congestion_level", "avg_speed_kmh"]], on="city")
df_summary = df_summary.merge(df_edu[["city", "graduation_rate", "total_schools"]], on="city")

# Add derived risk score
df_summary["risk_score"] = (
    (df_summary["aqi"] / 5) * 0.3 +
    (df_summary["congestion_level"]) * 0.3 +
    (1 - df_summary["graduation_rate"]) * 0.2 +
    (df_summary["pm2_5"] / 100) * 0.2
)
df_summary["risk_score"] = (df_summary["risk_score"] / df_summary["risk_score"].max()) * 100
df_summary["risk_level"] = pd.cut(df_summary["risk_score"], bins=3, labels=["Low", "Medium", "High"])

print("✅ Integrated city summary:")
df_summary

✅ Integrated city summary:


,city,aqi,pm2_5,pm10,congestion_level,avg_speed_kmh,graduation_rate,total_schools,risk_score,risk_level
0,Addis Ababa,4,16.731372,32.192541,0.69,57.62,0.821,586,75.735837,Medium
1,New York,5,95.258757,127.859990,0.20,27.25,0.837,407,85.543444,Medium
2,Los Angeles,4,93.431214,147.339963,0.75,41.54,0.851,365,100.000000,High
3,Chicago,1,69.121875,131.880226,0.67,38.41,0.931,494,60.593592,Low
4,Houston,4,79.395924,43.518376,0.46,16.22,0.723,219,86.874650,High
5,Phoenix,4,37.788322,93.744924,0.93,53.92,0.723,307,95.351690,High


In [13]:
m = folium.Map(location=[39.8283, -98.5795], zoom_start=4)

for _, row in df_summary.iterrows():
    color = {"Low": "green", "Medium": "orange", "High": "red"}[row["risk_level"]]
    location = df_noise[df_noise.city == row["city"]].iloc[0][["location_lat", "location_lon"]].values.tolist()

    folium.CircleMarker(
        location=location,
        radius=8,
        color=color,
        fill=True,
        fill_color=color,
        popup=f"{row['city']}: Risk {row['risk_level']} (Score: {row['risk_score']:.1f})"
    ).add_to(m)

print("🌍 Smart City Risk Map")
m


🌍 Smart City Risk Map


In [14]:
# Aggregate health data
health_agg = df_health.groupby(['city', 'date']).sum().reset_index()

fig = px.line(health_agg, x='date', y='respiratory_cases', color='city',
              title="Respiratory Cases Over Time by City",
              labels={"respiratory_cases": "Daily Cases"})
fig.show()

In [16]:
noise_pivot = df_noise.groupby(['city', 'hour'])['noise_dB'].mean().reset_index()
noise_pivot = noise_pivot.pivot(index="city", columns="hour", values="noise_dB")


fig = px.imshow(noise_pivot, text_auto=True, color_continuous_scale="Viridis",
                title="Average Noise Level (dB) by City and Hour of Day")
fig.update_layout(xaxis_title="Hour of Day", yaxis_title="City")
fig.show()

In [17]:
# Use PM2.5, hour, and crime as features
X = []
y = []

for _, row in df_traffic.iterrows():
    city = row["city"]
    pm25 = df_air[df_air.city == city]["pm2_5"].values[0]
    crime_avg = df_crime[df_crime.city == city]["crimes_reported"].mean()
    X.append([pm25, crime_avg, np.random.randint(0,24)])  # simulate hour
    y.append(row["congestion_level"])

X = np.array(X)
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor(n_estimators=100)
model.fit(X_train, y_train)
preds = model.predict(X_test)

print(f"✅ Traffic Prediction MAE: {mean_absolute_error(y_test, preds):.3f}")

✅ Traffic Prediction MAE: 0.211


In [18]:
# Predict PM2.5 from congestion and respiratory cases
X_aq = []
y_aq = []

for city in CITIES:
    congestion = df_traffic[df_traffic.city == city]["congestion_level"].values[0]
    resp_avg = df_health[df_health.city == city]["respiratory_cases"].mean()
    X_aq.append([congestion, resp_avg])
    y_aq.append(df_air[df_air.city == city]["pm2_5"].values[0])

X_aq = np.array(X_aq)
y_aq = np.array(y_aq)

aq_model = RandomForestRegressor()
aq_model.fit(X_aq, y_aq)

print("✅ Air Quality Prediction Model Trained")

✅ Air Quality Prediction Model Trained


In [19]:
%%writefile smart_city_dashboard.py

import streamlit as st
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import folium
from streamlit_folium import st_folium

# -------------------------------
# Mock Data (In real app, load from API or CSV)
# -------------------------------
CITIES = ["New York", "Los Angeles", "Chicago", "Houston", "Phoenix"]

df_air = pd.DataFrame({
    "city": CITIES,
    "aqi": np.random.randint(1, 6, 5),
    "pm2_5": np.random.uniform(5, 100, 5),
    "pm10": np.random.uniform(10, 150, 5),
    "no2": np.random.uniform(10, 80, 5)
})

df_traffic = pd.DataFrame({
    "city": CITIES,
    "congestion_level": np.random.uniform(0.3, 0.9, 5),
    "avg_speed_kmh": np.random.uniform(20, 60, 5)
})

df_summary = df_air[["city", "aqi", "pm2_5"]].merge(df_traffic[["city", "congestion_level"]], on="city")
df_summary["risk_score"] = (df_summary["aqi"] * 0.4 + df_summary["congestion_level"] * 0.6) * 20
df_summary["risk_level"] = pd.cut(df_summary["risk_score"], bins=3, labels=["Low", "Medium", "High"])

# -------------------------------
# Streamlit App
# -------------------------------
st.set_page_config(layout="wide", page_title="🏙️ Smart City Dashboard")

st.title("🏙️ Smart City Dashboard")
st.markdown("Real-time urban analytics for smarter cities")

# Sidebar Filters
st.sidebar.header("Filters")
selected_cities = st.sidebar.multiselect("Select Cities", CITIES, default=CITIES)

df_filtered = df_summary[df_summary.city.isin(selected_cities)]

# Metrics
st.header("📊 Key Metrics")
col1, col2, col3, col4 = st.columns(4)
col1.metric("Avg Air Quality Index", f"{df_filtered['aqi'].mean():.1f}")
col2.metric("Avg Congestion Level", f"{df_filtered['congestion_level'].mean():.2f}")
col3.metric("Avg PM2.5", f"{df_filtered['pm2_5'].mean():.1f} μg/m³")
col4.metric("High Risk Cities", (df_filtered["risk_level"] == "High").sum())

# Map
st.header("🌍 Risk Map")
m = folium.Map(location=[39.8283, -98.5795], zoom_start=4)
for _, row in df_filtered.iterrows():
    color = {"Low": "green", "Medium": "orange", "High": "red"}[row["risk_level"]]
    folium.CircleMarker(
        location=[39 + np.random.rand(), -98 + np.random.rand()],
        radius=10,
        color=color,
        fill=True,
        fill_color=color,
        popup=f"{row['city']}: {row['risk_level']} Risk"
    ).add_to(m)
st_folium(m, width=700)

# Charts
st.header("📈 Analytics")

fig1 = px.bar(df_filtered, x='city', y='congestion_level', color='risk_level',
              title="Traffic Congestion by City", color_discrete_map={"Low": "green", "Medium": "orange", "High": "red"})
st.plotly_chart(fig1, use_container_width=True)

fig2 = px.scatter(df_filtered, x='pm2_5', y='congestion_level', text='city', size='aqi',
                  title="Air Quality vs Traffic Congestion",
                  labels={"pm2_5": "PM2.5 (μg/m³)", "congestion_level": "Congestion Level"})
st.plotly_chart(fig2, use_container_width=True)

# Forecast
st.header("🔮 Forecast: Predicted PM2.5")
congestion_input = st.slider("Expected Avg Congestion Level", 0.0, 1.0, 0.5)
health_input = st.slider("Expected Avg Respiratory Cases", 10, 100, 30)

# Simple linear proxy (replace with real model in production)
predicted_pm25 = 5 + 80 * congestion_input + 0.5 * health_input
st.info(f"Predicted PM2.5 Level: **{predicted_pm25:.1f} μg/m³**")

st.markdown("---")
st.markdown("✅ Built with Python, Streamlit, and Plotly | Ready for **Streamlit Cloud** deployment")

Writing smart_city_dashboard.py


In [ ]:
# Install and use localtunnel
!npm install -g localtunnel

# Run Streamlit in background
!streamlit run smart_city_dashboard.py &

# Expose it publicly
!lt --port 8501

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧
changed 22 packages in 2s
⠇
⠇3 packages are looking for funding
⠇  run `npm fund` for details
⠇


  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.169.92.15:8501



In [ ]:
# 1. Set up ngrok
from pyngrok import ngrok
ngrok.set_auth_token("30VeMuVjsJx7DTu3rO641azl6u5_5ptd3372c9d7cg2jFp3r6")

# 2. Start Streamlit in background
!streamlit run smart_city_dashboard.py &

# 3. Open ngrok tunnel to localhost:8501
public_url = ngrok.connect(8501)
print(f"🌍 Your app is live at: {public_url}")